Change Colab runtime hardware gas pedal to **T4 GPU**

In [1]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 890.2 kB/s eta 0:00:00


In [5]:
!pip uninstall triton --yes

Found existing installation: triton 2.0.0
Uninstalling triton-2.0.0:
  Successfully uninstalled triton-2.0.0


In [1]:
!pip install triton==2.0.0.dev20221202

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 33.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cac

In [1]:
import triton
print(triton.__version__)

import transformers
print(transformers.__version__)

2.0.0
4.41.2


In [3]:
!git clone https://github.com/yuddecho/DNABERT-2-Finetune-1400.git

Cloning into 'DNABERT-2-Finetune-1400'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 27 (delta 3), reused 24 (delta 3), pack-reused 0
Receiving objects: 100% (27/27), 70.96 KiB | 14.19 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Filtering content: 100% (2/2), 446.64 MiB | 14.38 MiB/s, done.


In [ ]:
import os 

# merge, get model bin
bin_file_dir = 'pytorch_model_files'
if not os.path.exists(bin_file_dir):
    raise
    
num_files = 10
total_contents = bytes()

for i in range(num_files):
    file_path = f'/content/DNABERT-2-Finetune-1400/pytorch_model_{i}.bin'
    
    with open(file_path, 'rb') as infile:
        total_contents += infile.read()
      
# write
with open(f'/content/DNABERT-2-Finetune-1400/pytorch_model.bin', 'wb') as outfile:
    outfile.write(total_contents)
    
print('ok')

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel

model_id = '/content/DNABERT-2-Finetune-1400'

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModel.from_pretrained(model_id, trust_remote_code=True)
print(model)

model = model.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(4096, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertUnpadAttention(
          (self): BertUnpadSelfAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (mlp): BertGatedLinearUnitMLP(
          (gated_layers): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELU(approximate='none')
          

In [3]:
# test
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]

inputs = inputs.cuda()

hidden_states = model(inputs)[0] # [1, sequence_length, 768]

# embedding with mean pooling
embedding_mean = torch.mean(hidden_states[0], dim=0)
print(embedding_mean.shape) # expect to be 768

# embedding with max pooling
embedding_max = torch.max(hidden_states[0], dim=0)[0]
print(embedding_max.shape) # expect to be 768

torch.Size([768])
torch.Size([768])
